In [1]:
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import SparkSession

# Criando a sessão Spark
spark = SparkSession.builder \
    .appName("MinhaSessaoSpark") \
    .config("spark.some.config.option", "config-value") \
    .getOrCreate()

# Verificando se a sessão foi criada com sucesso
print("Sessão Spark criada com sucesso!")


Sessão Spark criada com sucesso!


In [3]:
candidato = spark.read.format("csv").option("header", "true").load("candidatos.csv")
colocado = spark.read.format("csv").option("header", "true").load("colocados.csv")
candidato_colocados=spark.read.format("csv").option("header", "true").load("candidatos_que_foram_colocados.csv")


In [7]:
geografia=spark.read.format("csv").option("header", "true").load("geografia.csv")
geografia.show(2)

+--------------------+----------------+---------------+----------+----------+
|     estabelecimento|total_candidatos|total_colocados|  latitude| longitude|
+--------------------+----------------+---------------+----------+----------+
|Escola Superior N...|             725|            157|   39.6945|  -8.13057|
|Escola Superior d...|            1092|            311|40.2185907|-8.4092581|
+--------------------+----------------+---------------+----------+----------+
only showing top 2 rows



In [5]:
candidato.show(2)

+---+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+---------+
|_c0|codigo_estabelecimento|codigo_curso|Nº Ordem (parcial)|        cc|                Nome| Nota|Opção|   PI|12º|10º/11º|     estabelecimento|              escola|    curso|regime_pos_laboral|regime_noturno|candidato|
+---+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+---------+
|  0|                   602|        9003|                 1|151(...)15|MARTA DO VALE MAR...|188.7|    6|177.0|195|    195|Universidade de É...|Escola de Ciência...|Agronomia|                 0|             0|        1|
|  1|                   602|        9003|                 2|156(...)35|FRANCISCO ISIDORO...|177.8|    1|181.0|176|    176|Un

In [6]:
colocado.show(2)

+---+----------------------+--------------------+--------------------+------------+-----------------+--------------------+----------+------------------+--------------+---------+
|_c0|codigo_estabelecimento|     estabelecimento|              escola|codigo_curso|            curso|                Nome|        cc|regime_pos_laboral|regime_noturno|candidato|
+---+----------------------+--------------------+--------------------+------------+-----------------+--------------------+----------+------------------+--------------+---------+
|  0|                   140|Universidade dos ...|Faculdade de Ciên...|        9022|Ciências Agrárias|BERNARDO COUTO MATOS|150(...)35|                 0|             0|        0|
|  1|                   140|Universidade dos ...|Faculdade de Ciên...|        9022|Ciências Agrárias| ISAC SILVA MENDONÇA|158(...)81|                 0|             0|        0|
+---+----------------------+--------------------+--------------------+------------+-----------------+---------

In [4]:
candidato_colocados.show(2)

+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+-----------+-------------+-----------+
|codigo_estabelecimento|codigo_curso|Nº Ordem (parcial)|        cc|                Nome| Nota|Opção|   PI|12º|10º/11º|     estabelecimento|              escola|    curso|regime_pos_laboral|regime_noturno|candidato_x|  tipo_ensino|candidato_y|
+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+-----------+-------------+-----------+
|                   602|        9003|                 2|156(...)35|FRANCISCO ISIDORO...|177.8|    1|181.0|176|    176|Universidade de É...|Escola de Ciência...|Agronomia|                 0|             0|          1|universitario|          0|
|                   602|    

## PERGUNTA 1

In [34]:
from pyspark.sql.functions import count

# Contagem por curso no DataFrame "candidato" com informações adicionais
candidato_por_curso = candidato.groupBy(
    "codigo_curso", 
    "curso",
    "codigo_estabelecimento", 
    "estabelecimento", 
    "escola", 
    "regime_pos_laboral", 
    "regime_noturno"
).agg(count("*").alias("total_candidatos"))

# Contagem por curso no DataFrame "colocado" com informações adicionais
colocado_por_curso = colocado.groupBy(
    "codigo_curso", 
    "curso",
    "codigo_estabelecimento", 
    "estabelecimento", 
    "escola", 
    "regime_pos_laboral", 
    "regime_noturno"
).agg(count("*").alias("total_colocados"))

# Exibindo os resultados
print("Distribuição de candidatos por curso:")
candidato_por_curso.show(truncate=False)

print("Distribuição de colocados por curso:")
colocado_por_curso.show(truncate=False)


Distribuição de candidatos por curso:
+------------+--------------------------------+----------------------+---------------------------------------+--------------------------------------------------------------------------+------------------+--------------+----------------+
|codigo_curso|curso                           |codigo_estabelecimento|estabelecimento                        |escola                                                                    |regime_pos_laboral|regime_noturno|total_candidatos|
+------------+--------------------------------+----------------------+---------------------------------------+--------------------------------------------------------------------------+------------------+--------------+----------------+
|9841        |Reabilitação Psicomotora        |1510                  |Universidade de Lisboa                 |Faculdade de Motricidade Humana                                           |0                 |0             |326             |
|9813        |

In [35]:
candidato_por_curso.count(), colocado_por_curso.count()

(1113, 1089)

In [36]:
resultado_join = candidato_por_curso.join(
    colocado_por_curso, 
    on=[
        "codigo_curso", 
        "curso",
        "codigo_estabelecimento", 
        "estabelecimento", 
        "escola", 
        "regime_pos_laboral", 
        "regime_noturno"
    ], 
    how="left"  # Fazendo um join do tipo 
)

# Mostrando o resultado do join
resultado_join.show(truncate=False)

+------------+--------------------------------+----------------------+---------------------------------------+--------------------------------------------------------------------------+------------------+--------------+----------------+---------------+
|codigo_curso|curso                           |codigo_estabelecimento|estabelecimento                        |escola                                                                    |regime_pos_laboral|regime_noturno|total_candidatos|total_colocados|
+------------+--------------------------------+----------------------+---------------------------------------+--------------------------------------------------------------------------+------------------+--------------+----------------+---------------+
|9841        |Reabilitação Psicomotora        |1510                  |Universidade de Lisboa                 |Faculdade de Motricidade Humana                                           |0                 |0             |326             |49   

In [23]:
resultado_join.count()

1113

In [37]:
from pyspark.sql.functions import col
linhas_vazias = resultado_join.filter(
    " OR ".join([f"{c} IS NULL" for c in resultado_join.columns])
)

# Mostrando as linhas com valores vazios
print("Linhas com valores vazios:")
linhas_vazias.show(truncate=False)

Linhas com valores vazios:
+------------+-----------------------------------------------+----------------------+---------------------------------------+--------------------------------------------------------------------+------------------+--------------+----------------+---------------+
|codigo_curso|curso                                          |codigo_estabelecimento|estabelecimento                        |escola                                                              |regime_pos_laboral|regime_noturno|total_candidatos|total_colocados|
+------------+-----------------------------------------------+----------------------+---------------------------------------+--------------------------------------------------------------------+------------------+--------------+----------------+---------------+
|9760        |Física e Química                               |602                   |Universidade de Évora                  |Escola de Ciências e Tecnologia                               

In [38]:
resultado_join_sem_nulos = resultado_join.fillna(0)

In [39]:
resultado_join_sem_nulos_pd=resultado_join_sem_nulos.toPandas()
resultado_join_sem_nulos_pd.to_csv('num_candidatos_vc_num_colocados.csv')

## PERGUNTA 4

In [31]:
from pyspark.sql.functions import avg, round

# Calculando a média e arredondando para 3 casas decimais
medias = candidato_colocados.groupBy(
    "codigo_estabelecimento",
    "codigo_curso",
    "estabelecimento",
    "escola",
    "curso",
    "regime_pos_laboral",
    "regime_noturno"
).agg(
    round(avg("Nota"), 3).alias("media_Nota"),
    round(avg("PI"), 3).alias("media_PI"),
    round(avg("12º"), 3).alias("media_12º"),
    round(avg("10º/11º"), 3).alias("media_10º/11º")
)

# Mostrando o resultado
print("Médias calculadas por agrupamento (arredondadas a 3 casas decimais):")
medias.show(truncate=False)


Médias calculadas por agrupamento (arredondadas a 3 casas decimais):
+----------------------+------------+-------------------------------------------+-------------------------------------------------------------+---------------------------------+------------------+--------------+----------+--------+---------+-------------+
|codigo_estabelecimento|codigo_curso|estabelecimento                            |escola                                                       |curso                            |regime_pos_laboral|regime_noturno|media_Nota|media_PI|media_12º|media_10º/11º|
+----------------------+------------+-------------------------------------------+-------------------------------------------------------------+---------------------------------+------------------+--------------+----------+--------+---------+-------------+
|3163                  |9147        |Instituto Politécnico de Viana do Castelo  |Escola Superior de Tecnologia e Gestão                       |Gestão              

In [32]:
medias.count()

1089

In [33]:
medias_pd=medias.toPandas()
medias_pd.to_csv('medias_colocados.csv')

## PERGUNTA 5

In [42]:
# Filtrando os candidatos com "Opção" igual a 1
candidatos_filtrados = candidato.filter(candidato["Opção"] == 1)

# Contando os registros por grupo
candidatos_1op = candidatos_filtrados.groupBy(
    "codigo_estabelecimento",
    "codigo_curso",
    "curso",
    "estabelecimento",
    "escola",
    "regime_pos_laboral",
    "regime_noturno"
).count()

# Renomeando a coluna 'count' para 'total_candidatos_1op'
candidatos_1op_renomeado = candidatos_1op.withColumnRenamed("count", "total_candidatos_1op")

# Mostrando o resultado
print("Contagem de registros por grupo:")
candidatos_1op_renomeado.show(truncate=False)


Contagem de registros por grupo:
+----------------------+------------+------------------------------------------------+-------------------------------------------+------------------------------------------------+------------------+--------------+--------------------+
|codigo_estabelecimento|codigo_curso|curso                                           |estabelecimento                            |escola                                          |regime_pos_laboral|regime_noturno|total_candidatos_1op|
+----------------------+------------+------------------------------------------------+-------------------------------------------+------------------------------------------------+------------------+--------------+--------------------+
|1508                  |9556        |Higiene Oral                                    |Universidade de Lisboa                     |Faculdade de Medicina Dentária                  |0                 |0             |41                  |
|3082                  |917

In [44]:
# Filtrando os candidatos com "Opção" igual a 1
candidatos_filtrados = candidato_colocados.filter(candidato_colocados["Opção"] == 1)

# Contando os registros por grupo
colocados_1op = candidatos_filtrados.groupBy(
    "codigo_estabelecimento",
    "codigo_curso",
    "curso",
    "estabelecimento",
    "escola",
    "regime_pos_laboral",
    "regime_noturno"
).count()

# Renomeando a coluna 'count' para 'total_candidatos_1op'
colocados_1op_renomeado = colocados_1op.withColumnRenamed("count", "total_colocados_1op")

# Mostrando o resultado
print("Contagem de registros por grupo:")
colocados_1op_renomeado.show(truncate=False)


Contagem de registros por grupo:
+----------------------+------------+---------------------------------+-------------------------------------------+-------------------------------------------------------------+------------------+--------------+-------------------+
|codigo_estabelecimento|codigo_curso|curso                            |estabelecimento                            |escola                                                       |regime_pos_laboral|regime_noturno|total_colocados_1op|
+----------------------+------------+---------------------------------+-------------------------------------------+-------------------------------------------------------------+------------------+--------------+-------------------+
|505                   |9181        |História                         |Universidade de Coimbra                    |Faculdade de Letras                                          |0                 |0             |44                 |
|3139                  |9164        |Ge

In [45]:
candidatos_1op_renomeado.count(), colocados_1op_renomeado.count()

(1077, 1077)

In [46]:
resultado_join = candidatos_1op_renomeado.join(
    colocados_1op_renomeado, 
    on=[
        "codigo_curso", 
        "curso",
        "codigo_estabelecimento", 
        "estabelecimento", 
        "escola", 
        "regime_pos_laboral", 
        "regime_noturno"
    ], 
    how="inner"  # Fazendo um join do tipo 
)

# Mostrando o resultado do join
resultado_join.show(truncate=False)

+------------+---------------------------------+----------------------+-------------------------------------------+-------------------------------------------------------------+------------------+--------------+--------------------+-------------------+
|codigo_curso|curso                            |codigo_estabelecimento|estabelecimento                            |escola                                                       |regime_pos_laboral|regime_noturno|total_candidatos_1op|total_colocados_1op|
+------------+---------------------------------+----------------------+-------------------------------------------+-------------------------------------------------------------+------------------+--------------+--------------------+-------------------+
|9181        |História                         |505                   |Universidade de Coimbra                    |Faculdade de Letras                                          |0                 |0             |95                  |44       

In [49]:
df_proporcao = resultado_join.withColumn(
    "proporcao_colocados",
    round(((col("total_colocados_1op") / col("total_candidatos_1op"))*100),1)
)

# Mostrando o resultado
df_proporcao.show(truncate=False)

+------------+---------------------------------+----------------------+-------------------------------------------+-------------------------------------------------------------+------------------+--------------+--------------------+-------------------+-------------------+
|codigo_curso|curso                            |codigo_estabelecimento|estabelecimento                            |escola                                                       |regime_pos_laboral|regime_noturno|total_candidatos_1op|total_colocados_1op|proporcao_colocados|
+------------+---------------------------------+----------------------+-------------------------------------------+-------------------------------------------------------------+------------------+--------------+--------------------+-------------------+-------------------+
|9181        |História                         |505                   |Universidade de Coimbra                    |Faculdade de Letras                                          |0   

In [50]:
df_proporcao_pd=df_proporcao.toPandas()
df_proporcao_pd.to_csv('num5.csv')

## PERGUNTA 7

In [ ]:
from pyspark.sql import functions as F

df_filtrado = candidato_colocados.filter(F.col("opção") != 1)
colunas_desejadas = ["Nome", "cc", "Opção", "estabelecimento", "curso", "escola", "codigo_estabelecimento", "codigo_curso"]
colocados_n_1op = df_filtrado.select(*colunas_desejadas)

colocados_n_1op.show()

+--------------------+----------+-----+--------------------+-------------------+--------------------+----------------------+------------+
|                Nome|        cc|Opção|     estabelecimento|              curso|              escola|codigo_estabelecimento|codigo_curso|
+--------------------+----------+-----+--------------------+-------------------+--------------------+----------------------+------------+
|JORGE DINIS DE JE...|307(...)84|    2|Universidade de É...|          Agronomia|Escola de Ciência...|                   602|        9003|
|MADALENA DUARTE V...|153(...)42|    2|Universidade de É...|          Agronomia|Escola de Ciência...|                   602|        9003|
|JOÃO ANTÓNIO GOME...|303(...)27|    2|Universidade de É...|          Agronomia|Escola de Ciência...|                   602|        9003|
|JOSÉ LUÍS BRITO M...|307(...)51|    6|Universidade de É...|          Agronomia|Escola de Ciência...|                   602|        9003|
|LUCAS SANTIAGO VE...|301(...)50| 

In [ ]:
colunas_para_renomear = ["Opção", "estabelecimento", "curso", "escola", "codigo_estabelecimento", "codigo_curso"]
for coluna in colunas_para_renomear:
    colocados_n_1op = colocados_n_1op.withColumnRenamed(coluna, f"A{coluna}")

In [ ]:
colocados_n_1op.show(2)s

+--------------------+----------+------+--------------------+---------+--------------------+-----------------------+-------------+
|                Nome|        cc|AOpção|    Aestabelecimento|   Acurso|             Aescola|Acodigo_estabelecimento|Acodigo_curso|
+--------------------+----------+------+--------------------+---------+--------------------+-----------------------+-------------+
|JORGE DINIS DE JE...|307(...)84|     2|Universidade de É...|Agronomia|Escola de Ciência...|                    602|         9003|
|MADALENA DUARTE V...|153(...)42|     2|Universidade de É...|Agronomia|Escola de Ciência...|                    602|         9003|
+--------------------+----------+------+--------------------+---------+--------------------+-----------------------+-------------+
only showing top 2 rows



In [13]:
colocados_n_1op.count()

21922

In [ ]:
nomes_cc_df = colocados_n_1op.select("Nome", "cc")
df_resultado = candidato.filter(
    (candidato["Opção"] == 1) & 
    (candidato["Nome"].isin([row["Nome"] for row in nomes_cc_df.collect()])) &
    (candidato["cc"].isin([row["cc"] for row in nomes_cc_df.collect()]))
)

df_resultado.show()

+----+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+--------------------+------------------+--------------+---------+
| _c0|codigo_estabelecimento|codigo_curso|Nº Ordem (parcial)|        cc|                Nome| Nota|Opção|   PI|12º|10º/11º|     estabelecimento|              escola|               curso|regime_pos_laboral|regime_noturno|candidato|
+----+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+--------------------+------------------+--------------+---------+
| 660|                   602|        9011|               204|156(...)81|GUSTAVO MARTINS S...|119.7|    1|108.0|126|    126|Universidade de É...|Escola de Ciência...|            Biologia|                 0|             0|        1|
|1075|                   602|        9143|               121|309(...)86|  MA

In [10]:
colunas_desejadas = ["Nome", "cc", "Opção", "estabelecimento", "curso", "escola", "codigo_estabelecimento", "codigo_curso"]
candidatos_1op = df_resultado.select(*colunas_desejadas)

In [12]:
candidatos_1op.show(2)

+--------------------+----------+-----+--------------------+---------+--------------------+----------------------+------------+
|                Nome|        cc|Opção|     estabelecimento|    curso|              escola|codigo_estabelecimento|codigo_curso|
+--------------------+----------+-----+--------------------+---------+--------------------+----------------------+------------+
|GUSTAVO MARTINS S...|156(...)81|    1|Universidade de É...| Biologia|Escola de Ciência...|                   602|        9011|
|  MARTIM JARDIM DIAS|309(...)86|    1|Universidade de É...|Geografia|Escola de Ciência...|                   602|        9143|
+--------------------+----------+-----+--------------------+---------+--------------------+----------------------+------------+
only showing top 2 rows



In [1]:
candidatos_1op.count()

NameError: name 'candidatos_1op' is not defined

In [11]:
df_join = candidatos_1op.join(
    colocados_n_1op, 
    on=["Nome", "cc"],  # As colunas de junção
    how="inner"  # Tipo de join: "inner" significa que só serão mantidas as linhas com correspondência em ambos os DataFrames
)

# Mostrando o resultado
df_join.show()

+--------------------+----------+-----+--------------------+--------------------+--------------------+----------------------+------------+------+--------------------+--------------------+--------------------+-----------------------+-------------+
|                Nome|        cc|Opção|     estabelecimento|               curso|              escola|codigo_estabelecimento|codigo_curso|AOpção|    Aestabelecimento|              Acurso|             Aescola|Acodigo_estabelecimento|Acodigo_curso|
+--------------------+----------+-----+--------------------+--------------------+--------------------+----------------------+------------+------+--------------------+--------------------+--------------------+-----------------------+-------------+
|GUSTAVO MARTINS S...|156(...)81|    1|Universidade de É...|            Biologia|Escola de Ciência...|                   602|        9011|     3|Instituto Politéc...|Tecnologia e Gest...|Escola Superior A...|                   3061|         L024|
|  MARTIM JA

In [12]:
df_join.count()

21922

In [16]:
df_join_pd=df_join.toPandas()
df_join_pd.to_csv('op_unix_cursoy.csv')

## PERGUNTA 10

In [10]:
candidato_colocados.show(2)

+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+-----------+-------------+-----------+
|codigo_estabelecimento|codigo_curso|Nº Ordem (parcial)|        cc|                Nome| Nota|Opção|   PI|12º|10º/11º|     estabelecimento|              escola|    curso|regime_pos_laboral|regime_noturno|candidato_x|  tipo_ensino|candidato_y|
+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+-----------+-------------+-----------+
|                   602|        9003|                 2|156(...)35|FRANCISCO ISIDORO...|177.8|    1|181.0|176|    176|Universidade de É...|Escola de Ciência...|Agronomia|                 0|             0|          1|universitario|          0|
|                   602|    

In [27]:
from pyspark.sql import functions as F

candidato_colocados_regime = candidato_colocados.withColumn(
    "regime",
    F.when((F.col("regime_pos_laboral") == 0) & (F.col("regime_noturno") == 0), "Diurno")
     .when(F.col("regime_pos_laboral") == 1, "Pós-Laboral")
     .when(F.col("regime_noturno") == 1, "Noturno")
     .otherwise("Indefinido")  
)

# Agrupar por curso e regime e calcular a média da coluna Nota
result = candidato_colocados_regime.groupBy("curso", "regime", "estabelecimento").agg(
    F.min("Nota").alias("Nota_minima")
)

# Mostrar o resultado
result.show()

+--------------------+-----------+--------------------+-----------+
|               curso|     regime|     estabelecimento|Nota_minima|
+--------------------+-----------+--------------------+-----------+
|Administração Púb...|     Diurno|Instituto Politéc...|      109.0|
|Administração Púb...|     Diurno|Instituto Politéc...|      127.9|
|Administração Púb...|     Diurno|Universidade de A...|      120.2|
|Administração Púb...|     Diurno|Universidade de L...|      140.0|
|Administração Púb...|     Diurno|Universidade do M...|      160.2|
|Administração Púb...|Pós-Laboral|Universidade de L...|      135.5|
|Administração Púb...|Pós-Laboral|Universidade de L...|      128.5|
|Administração Púb...|     Diurno|Universidade de C...|      143.0|
|Administração de ...|     Diurno|Instituto Politéc...|      115.4|
|           Agronomia|     Diurno|Instituto Politéc...|      124.4|
|           Agronomia|     Diurno|Instituto Politéc...|      122.4|
|           Agronomia|     Diurno|Instituto Poli

In [29]:
result_pd=result.toPandas()
result_pd.to_csv('nota_corte_por_regimes.csv')

## PERGUNTA 11

In [21]:
candidato.show(2)

+---+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+---------+
|_c0|codigo_estabelecimento|codigo_curso|Nº Ordem (parcial)|        cc|                Nome| Nota|Opção|   PI|12º|10º/11º|     estabelecimento|              escola|    curso|regime_pos_laboral|regime_noturno|candidato|
+---+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+---------+
|  0|                   602|        9003|                 1|151(...)15|MARTA DO VALE MAR...|188.7|    6|177.0|195|    195|Universidade de É...|Escola de Ciência...|Agronomia|                 0|             0|        1|
|  1|                   602|        9003|                 2|156(...)35|FRANCISCO ISIDORO...|177.8|    1|181.0|176|    176|Un

In [24]:
from pyspark.sql import functions as F

opcao_1_df = candidato.filter(F.col("Opção") == 1)

frequencia_estabelecimentos = opcao_1_df.groupBy("estabelecimento").agg(
    F.count("*").alias("frequencia")
)

frequencia_estabelecimentos = frequencia_estabelecimentos.orderBy(F.desc("frequencia"))

# Mostrar o resultado
frequencia_estabelecimentos.show(truncate=False)


+-------------------------------------------+----------+
|estabelecimento                            |frequencia|
+-------------------------------------------+----------+
|Universidade de Lisboa                     |9167      |
|Universidade do Porto                      |8069      |
|Universidade Nova de Lisboa                |4104      |
|Instituto Politécnico do Porto             |3910      |
|Universidade de Coimbra                    |3884      |
|Universidade do Minho                      |3836      |
|Instituto Politécnico de Lisboa            |2734      |
|Universidade de Aveiro                     |2730      |
|ISCTE                                      |2308      |
|Instituto Politécnico de Coimbra           |1996      |
|Instituto Politécnico de Leiria            |1745      |
|Universidade do Algarve                    |1582      |
|Universidade de Trás-os-Montes e Alto Douro|1390      |
|Universidade da Beira Interior             |1143      |
|Instituto Politécnico de Setúb

In [31]:
frequencia_estabelecimentos_pd=frequencia_estabelecimentos.toPandas()
frequencia_estabelecimentos_pd.to_csv('freq_1op_estabelecimento.csv')

## PERGUNTA 12

In [30]:
from pyspark.sql import functions as F

opcao_1_df = candidato.filter(F.col("Opção") == 1)

frequencia_cursos = opcao_1_df.groupBy("curso").agg(
    F.count("*").alias("frequencia")
)

frequencia_cursos = frequencia_cursos.orderBy(F.desc("frequencia"))

# Mostrar o resultado
frequencia_cursos.show(truncate=False)

+------------------------------------------+----------+
|curso                                     |frequencia|
+------------------------------------------+----------+
|Gestão                                    |3545      |
|Engenharia Informática                    |2747      |
|Psicologia                                |2706      |
|Direito                                   |2610      |
|Medicina                                  |2193      |
|Enfermagem                                |1927      |
|Educação Básica                           |1322      |
|Engenharia Mecânica                       |1278      |
|Arquitetura                               |1185      |
|Economia                                  |1134      |
|Fisioterapia                              |957       |
|Ciências do Desporto                      |936       |
|Ciências da Comunicação                   |895       |
|Marketing                                 |843       |
|Ciências Farmacêuticas                    |777 

In [32]:
frequencia_cursos_pd=frequencia_cursos.toPandas()
frequencia_cursos_pd.to_csv('freq_1op_cursos.csv')

## PERGUNTA 14

In [38]:
candidato_colocados.show(2)

+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+-----------+-------------+-----------+
|codigo_estabelecimento|codigo_curso|Nº Ordem (parcial)|        cc|                Nome| Nota|Opção|   PI|12º|10º/11º|     estabelecimento|              escola|    curso|regime_pos_laboral|regime_noturno|candidato_x|  tipo_ensino|candidato_y|
+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+--------------------+---------+------------------+--------------+-----------+-------------+-----------+
|                   602|        9003|                 2|156(...)35|FRANCISCO ISIDORO...|177.8|    1|181.0|176|    176|Universidade de É...|Escola de Ciência...|Agronomia|                 0|             0|          1|universitario|          0|
|                   602|    

In [ ]:
from pyspark.sql import functions as F

candidatos_nota = candidato.filter(F.col("Opção") == 1)
num_candidatos_maior_17 = candidatos_nota.filter(F.col("Nota") >= 170).count()
num_candidatos_menor_17 = candidatos_nota.filter(F.col("Nota") < 170).count()

colocados_nota = candidato_colocados.filter(F.col("Opção") == 1)
num_colocados_maior_17 = colocados_nota.filter(F.col("Nota") >= 170).count()
num_colocados_menor_17 = colocados_nota.filter(F.col("Nota") < 170).count()

data = [
    ("Maior que 17", 
     float(num_colocados_maior_17 )/ num_candidatos_maior_17 * 100 if num_candidatos_maior_17 > 0 else 0),
    ("Menor que 17", 
     float(num_colocados_menor_17 )/ num_candidatos_menor_17 * 100 if num_candidatos_menor_17 > 0 else 0)
]

data


[('Maior que 17', 73.24357229821672), ('Menor que 17', 40.17772565293879)]

In [38]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# Definir os filtros
estabelecimento = "Universidade do Minho"
curso = "Medicina"
#nome_parcial = "ANA CLARA"  # Pode ser qualquer parte do nome

# Filtragem usando .filter() e col() para acessar as colunas, utilizando 'contains' para nome parcial
filtro_df = candidato_colocados.filter(
    (col("estabelecimento") == estabelecimento) &
    (col("curso") == curso) 
    #(col("Nome").contains(nome_parcial))  # Filtra por nome parcialmente
)

# Conversão para garantir que a ordenação seja feita corretamente, caso a coluna seja string
filtro_df = filtro_df.withColumn("Nº Ordem (parcial)", col("Nº Ordem (parcial)").cast("int"))

# Usar o Window para recalcular a ordem dos candidatos
windowSpec = Window.partitionBy("estabelecimento", "curso").orderBy(col("Nº Ordem (parcial)"))

# Adicionar uma coluna com a ordem recalculada
filtro_df = filtro_df.withColumn("Nova Ordem", row_number().over(windowSpec))

# Exibir os resultados ordenados
filtro_df.show()


+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+------+--------+------------------+--------------+-----------+-------------+-----------+----------+
|codigo_estabelecimento|codigo_curso|Nº Ordem (parcial)|        cc|                Nome| Nota|Opção|   PI|12º|10º/11º|     estabelecimento|escola|   curso|regime_pos_laboral|regime_noturno|candidato_x|  tipo_ensino|candidato_y|Nova Ordem|
+----------------------+------------+------------------+----------+--------------------+-----+-----+-----+---+-------+--------------------+------+--------+------------------+--------------+-----------+-------------+-----------+----------+
|                  1000|        9813|                 9|310(...)40|AFONSO MIGUEL PER...|197.5|    1|195.0|200|    200|Universidade do M...|     D|Medicina|                 0|             0|          1|universitario|          0|         1|
|                  1000|        9813|       

In [39]:
filtro_df=filtro_df.toPandas()
filtro_df.to_csv('MARIANA_FOFOQUEIRA.csv')

In [5]:
spark.stop()